# My Solution:

In [32]:
from collections import deque
import random, time

def get_neighbors():
    neighbors = {}
    for r in range(9):
        for c in range(9):
            cell = (r, c)
            cells = set()
            for i in range(9):
                cells.add((r, i))
                cells.add((i, c))
            start_row, start_col = 3 * (r // 3), 3 * (c // 3)
            for i in range(3):
                for j in range(3):
                    cells.add((start_row + i, start_col + j))
            cells.discard(cell)
            neighbors[cell] = cells
    return neighbors

def parse_puzzle(puzzle_str):
    puzzle = {}
    for i in range(81):
        row, col = divmod(i, 9)
        val = int(puzzle_str[i])
        puzzle[(row, col)] = [val] if val != 0 else list(range(1, 10))
    return puzzle

def ac3(puzzle, neighbors):
    queue = deque([(cell, n) for cell in puzzle for n in neighbors[cell]])
    while queue:
        xi, xj = queue.popleft()
        if revise(puzzle, xi, xj):
            if not puzzle[xi]:
                return False
            for xk in neighbors[xi]:
                if xk != xj:
                    queue.append((xk, xi))
    return True

def revise(puzzle, xi, xj):
    revised = False
    to_remove = []
    for val in puzzle[xi]:
        if len(puzzle[xj]) == 1 and puzzle[xj][0] == val:
            to_remove.append(val)
            revised = True
    for val in to_remove:
        puzzle[xi].remove(val)
    return revised

def is_valid(puzzle, cell, val, neighbors):
    return all(not (len(puzzle[n]) == 1 and puzzle[n][0] == val) for n in neighbors[cell])

def select_unassigned_var(puzzle):
    unassigned = [(len(puzzle[k]), k) for k in puzzle if len(puzzle[k]) > 1]
    return min(unassigned)[1] if unassigned else None

def backtrack(puzzle, neighbors):
    if all(len(puzzle[k]) == 1 for k in puzzle):
        return puzzle
    var = select_unassigned_var(puzzle)
    for val in puzzle[var]:
        if is_valid(puzzle, var, val, neighbors):
            saved = puzzle.copy()
            puzzle[var] = [val]
            if ac3(puzzle, neighbors):
                result = backtrack(puzzle, neighbors)
                if result:
                    return result
            puzzle = saved
    return None

def solve(puzzle_str):
    neighbors = get_neighbors()
    puzzle = parse_puzzle(puzzle_str)
    if not ac3(puzzle, neighbors):
        return None
    result = backtrack(puzzle, neighbors)
    return ''.join(str(result[(r, c)][0]) for r in range(9) for c in range(9)) if result else None

# Timing and test
def generate_easy_puzzle():
    base = '530070000600195000098000060800060003400803001700020006060000280000419005000080079'
    return base  # fixed easy puzzle

start = time.time()
puzzle = generate_easy_puzzle()
solution = solve(puzzle)
end = time.time()

print("AC3 + Backtracking Solver")
print("Time Taken:", round(end - start, 4), "seconds")
print("Solution:", solution)


AC3 + Backtracking Solver
Time Taken: 0.0086 seconds
Solution: 534678912672195348198342567859761423426853791713924856961537284287419635345286179


# OR-TOOLS Version:

In [33]:
from ortools.sat.python import cp_model
import time

def solve_sudoku_ortools(puzzle_str):
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    grid = [[model.NewIntVar(1, 9, f'cell_{r}_{c}') for c in range(9)] for r in range(9)]

    for r in range(9):
        for c in range(9):
            val = int(puzzle_str[r * 9 + c])
            if val != 0:
                model.Add(grid[r][c] == val)

    for i in range(9):
        model.AddAllDifferent([grid[i][j] for j in range(9)])
        model.AddAllDifferent([grid[j][i] for j in range(9)])

    for box_r in range(3):
        for box_c in range(3):
            model.AddAllDifferent([
                grid[r][c]
                for r in range(box_r * 3, box_r * 3 + 3)
                for c in range(box_c * 3, box_c * 3 + 3)
            ])

    status = solver.Solve(model)
    if status == cp_model.FEASIBLE:
        return ''.join(str(solver.Value(grid[r][c])) for r in range(9) for c in range(9))
    else:
        return "No solution"

# Timing
puzzle = '530070000600195000098000060800060003400803001700020006060000280000419005000080079'
start = time.time()
solution = solve_sudoku_ortools(puzzle)
end = time.time()

print("OR-Tools CP-SAT Solver")
print("Time Taken:", round(end - start, 4), "seconds")
print("Solution:", solution)

OR-Tools CP-SAT Solver
Time Taken: 0.0262 seconds
Solution: No solution


# CHAT-GPT/Github Version:

In [34]:
from collections import deque
import time

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(A, B):
    return [a + b for a in A for b in B]

cells = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
col_units = [cross(rows, c) for c in cols]
box_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + col_units + box_units
units = {s: [u for u in unitlist if s in u] for s in cells}
peers = {s: set(sum(units[s],[])) - {s} for s in cells}

def parse_grid(grid):
    values = {s: '123456789' for s in cells}
    for s, d in zip(cells, grid):
        if d in '123456789' and not assign(values, s, d):
            return False
    return values

def assign(values, s, d):
    other_values = values[s].replace(d, '')
    return values if all(eliminate(values, s, d2) for d2 in other_values) else False

def eliminate(values, s, d):
    if d not in values[s]:
        return values
    values[s] = values[s].replace(d, '')
    if len(values[s]) == 0:
        return False
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    for unit in units[s]:
        dplaces = [s2 for s2 in unit if d in values[s2]]
        if len(dplaces) == 0:
            return False
        elif len(dplaces) == 1 and not assign(values, dplaces[0], d):
            return False
    return values

def ac3(values):
    queue = deque((s, p) for s in cells for p in peers[s])
    while queue:
        xi, xj = queue.popleft()
        if revise(values, xi, xj):
            if len(values[xi]) == 0:
                return False
            for xk in peers[xi] - {xj}:
                queue.append((xk, xi))
    return values

def revise(values, xi, xj):
    revised = False
    for x in values[xi]:
        if len(values[xj]) == 1 and values[xj] == x:
            values[xi] = values[xi].replace(x, '')
            revised = True
    return revised

def is_solved(values):
    return all(len(values[s]) == 1 for s in cells)

def search(values):
    if values is False:
        return False
    if is_solved(values):
        return values
    n, s = min((len(values[s]), s) for s in cells if len(values[s]) > 1)
    for d in values[s]:
        result = search(assign(values.copy(), s, d))
        if result:
            return result
    return False

def solve(grid):
    return search(ac3(parse_grid(grid)))

def grid_to_string(values):
    return ''.join(values[s] if len(values[s]) == 1 else '.' for s in cells)

# Timing
puzzle = '530070000600195000098000060800060003400803001700020006060000280000419005000080079'
start = time.time()
solved = solve(puzzle)
end = time.time()

print("ChatGPT Constraint Propagation Solver")
print("Time Taken:", round(end - start, 4), "seconds")
print("Solution:", grid_to_string(solved))


ChatGPT Constraint Propagation Solver
Time Taken: 0.0035 seconds
Solution: 534678912672195348198342567859761423426853791713924856961537284287419635345286179


# Theoretical answeres:

### 1-My Custom Version:
-Uses Backtracking + Forward Checking or AC-3 (Constraint Propagation).

-Focus is on understanding and implementing the logic manually.

-It has control over:

    Constraint propagation

    Variable ordering (e.g., MRV or not)

    Verbose/debug logs

### 2-Google OR-Tools:
-Uses advanced CSP solvers and optimization engines internally.

-Handles variable/value ordering heuristics, backtracking, and propagation using optimized C++ backend.

-It is extremely fast, efficient, and scalable.

-Best for industrial-level applications.

### 3-GitHub/ChatGPT-style Sudoku Solvers:
-Generally backtracking + optional enhancements:

    MRV (Minimum Remaining Values)

    Forward checking
    
    Some use recursion, some iterative

-Mid-performance: better than naive, but worse than OR-Tools.

-Still Python-based — slower than compiled OR-Tools.

# Improvements:

-MRV Heuristic (Minimum Remaining Values):
    
    Choose the cell with the fewest legal values first.

-Degree Heuristic:
    
    If two variables tie in MRV, choose the one with the most constraints on remaining variables.

-Forward Checking:
    
    Eliminate values from neighbors when a value is assigned.

-AC-3 before/during search:
    
    Prune domain values using Arc Consistency.

-Use a 1D representation (dict) instead of 2D list for better domain checking.

-Use numpy for faster array operations.

-Memoization or cache domain reductions in repeated searches.